In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd


from pymongo import MongoClient



###########################
# Data Manipulation / Model
###########################
from Create_Read import AnimalShelter
username = "aacuser"
password = "1qaz!QAZ"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')



image_filename = 'Grazioso Salvare Logo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Div(
        html.Center(html.H5('Bauer', style={'textAlign':'center', 'float':'right', 
                                                 'marginTop':'7%'}))),

    html.Div(
        html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 alt='customer image', height=150),
        style={'display':'inline-block', 'marginLeft':'45%'}),
    html.Hr(),

    html.Div(className='row',
             style={'display' : 'flex'},
                  children=[
                          dcc.Dropdown(
                              id ='Animal_Filter', 
                              options = [{'label':'Reset', 'value':'Default'}, 
                                         {'label':'Water Rescue Breeds', 'value':'Water'}, 
                                         {'label':'Wilderness Rescue Breeds', 'value': 'Wild'},
                                         {'label':'Disaster Rescue Breeds', 'value': 'Disaster'}],
                              value = 'Start',
                              placeholder = 'Filter',
                              style = {'width' : '50%'}),
                  ]),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        style_table={'overflowX':'auto'},
        style_cell={'height':'auto','minWidth':'150px', 'maxwidth':'300px','width':'150px',
                    'whiteSpace':'normal','textAlign' : 'left'},        
        style_header={'textAlign':'left', 'fontWeight' : 'bold'},
        data=df.to_dict('records'),
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Hr(),
    html.Br(), 
    
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    
    html.Hr(),

])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


    
@app.callback(Output('datatable-interactivity',"data"), 
              [Input('Animal_Filter', 'value')])
def filter_table(filter_type):
    if (str(filter_type) == "Default" or str(filter_type) == "Start"):
        df = pd.DataFrame(list(shelter.read({})))
    elif (str(filter_type) == "Water"):
        df = pd.DataFrame(list(shelter.read({"breed":{"$in":['Labrador Retriever','Chesa Bay Retr','Newfoundland']},
                                             "sex_upon_outcome":'Intact Female', 
                                                "age_upon_outcome_in_weeks":{"$gte":26,"$lt":156}})))                    
    elif (str(filter_type) == "Wild"):
        df = pd.DataFrame(list(shelter.read({"breed":{"$in":['German Shepherd','Alaskan Malamute',
                                                            'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
                                             "sex_upon_outcome":'Intact Male', 
                                                "age_upon_outcome_in_weeks":{"$gte":26,"$lt":156}})))
    elif (str(filter_type) == "Disaster") :
        df = pd.DataFrame(list(shelter.read({"breed":{"$in":['Doberman Pinsch','German Shepherd', 
                                                             'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                                             "sex_upon_outcome":'Intact Male', 
                                                "age_upon_outcome_in_weeks":{"$gte":20,"$lt":300}})))
    return df.to_dict('records')


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"), 
     Input('datatable-interactivity', "selected_rows")])

def update_map(viewData, selected_row):
    
    if selected_row == []:
        dff = pd.DataFrame.from_dict(viewData)
        return [
        dl.Map(style={'width':'1000px', 'height': '300px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                    ])
                ])
            ])
        ]
    else:
        dff = pd.DataFrame.from_dict(viewData)
        return [
        dl.Map(style={'width':'1000px', 'height': '300px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[selected_row[0],13],dff.iloc[selected_row[0],14]], children=[
                dl.Tooltip(dff.iloc[selected_row[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selected_row[0],9])
                    ])
                ])
            ])
        ]
    
@app.callback(
    Output('graph-id', "children"), 
    [Input('datatable-interactivity', "derived_viewport_data")])
def generate_chart(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(data_frame=dff, values=None, names='breed')
    return dcc.Graph(figure = px.pie(data_frame=dff, values=None, names='breed'))

app
